In [1]:
import json
import re

import pandas as pd
import numpy as np

In [2]:
make_varmix = False  # 10/18/2023

if make_varmix:
    df = pd.read_excel('solmix.xlsx', index_col=0)[['G98D']]
    df = df.rename(columns={'G98D': 'GS98'})
    elements = list(df.index.values[5:])

In [3]:
# from kap/aesopus_maker.py

alpha_elements = ['O', 'Ne', 'Mg', 'Si', 'S', 'Ar', 'Ca']

def CFe_NFe_calc(FeH):
    CFe = 0.268 * FeH**2 + 0.0258 * FeH - 0.00983
    NFe = 0.373 * FeH**2 + 0.373  * FeH + 0.0260
    return CFe, NFe

In [4]:
# from chem/solmix.ipynb

with open('weights.json', 'r') as f:
    weights = json.load(f)

OP_elements_main = ['H', 'He', 'C', 'N', 'O', 'Ne', 'Na', 'Mg', 'Al', 'Si', 'S', 'Ca', 'Ar', 'Cr', 'Mn', 'Fe', 'Ni']
OP_elements_plus = ['P', 'Cl', 'K', 'Ti']
OP_elements = OP_elements_main + OP_elements_plus

def is_in_OP(elem: str) -> bool:
    return elem in OP_elements

In [5]:
# from chem/solmix.ipynb

def calc_mass_fractions(col: str) -> None:
    X = 10.0 ** df.at['H',  col] * weights['H' ]
    Y = 10.0 ** df.at['He', col] * weights['He']

    Z, OP = 0.0, 0.0
    for metal in elements[2:]:
        fraction = 10.0 ** df.at[metal, col] * weights[metal]
        Z += fraction
        if is_in_OP(metal): OP += fraction
    total = X + Y + Z

    df.at['X_frac', col] = X / total
    df.at['Y_frac', col] = Y / total
    df.at['Z_frac', col] = Z / total
    df.at['Z/X',    col] = (Z / X) if X else 0.0
    df.at['OP/Z',   col] = OP / Z

In [6]:
if make_varmix:
    for AFe in [+0.1]:
        col = f'A{round(AFe, 1):+.1f}'
        df[col] = df['GS98']

        for elem in alpha_elements:
            df.at[elem, col] += AFe
        calc_mass_fractions(col)

In [7]:
if make_varmix:
    for FeH in np.linspace(-0.45, +0.45, 7):
        col = f'CN{round(FeH+1e-6, 2):+.2f}'
        df[col] = df['GS98']

        CFe, NFe = CFe_NFe_calc(FeH)
        df.at['C', col] += CFe
        df.at['N', col] += NFe
        calc_mass_fractions(col)

In [8]:
if make_varmix:
    df.to_excel('varmix.xlsx')
    del df

In [9]:
df = pd.read_excel('varmix.xlsx', index_col=0)
# elements = list(df.index.values[5:])

In [10]:
pp_and_cno_extras = ['h1', 'h2', 'he3', 'he4', 'li7', 'be7', 'b8',
                     'c12', 'c13', 'n13', 'n14', 'n15', 'o14', 'o15', 'o16', 'o17', 'o18',
                     'f17', 'f18', 'f19', 'ne18', 'ne19', 'ne20', 'mg22', 'mg24']

In [11]:
def get_elem_and_a(iso: str) -> (str, int):
    m = re.match('([a-z]+)([0-9]+)', iso)
    elem = m.group(1).capitalize()
    a = m.group(2)
    return elem, a

if False:  # test get_elem_and_a, 10/18/2023
    for iso in pp_and_cno_extras:
        print(iso, *get_elem_and_a(iso))

In [12]:
# parse $MESA_DIR/chem/data/lodders03.data
# different from chem/solmix.ipynb

with open('lodders03.data') as f:
    lines = f.readlines()

lodders03 = {}

for line in lines:
    if line.startswith('!'): continue
    z, elem, a, frac, n = line.split()

    if elem not in lodders03:
        lodders03[elem] = {}
    lodders03[elem][a] = float(frac)/100.0

if False:  # test lodders03, 10/18/2023
    for elem, fracs in lodders03.items():
        for a, frac in fracs.items():
            iso = f'{elem.lower()}{a}'
            print(elem, a, frac, isotopes[iso])

In [13]:
# from chem/solmix.ipynb

with open('isotopes.json', 'r') as f:
    isotopes = json.load(f)

In [14]:
calc_mass_fracs = False  # never used, 10/18/2023

if calc_mass_fracs:
    mass_fracs = {}

    for elem, num_fracs in lodders03.items():
        mass_fracs[elem] = {}

        for a, num_frac in num_fracs.items():
            iso = f'{elem.lower()}{a}'
            mass_fracs[elem][a] = num_frac * isotopes[iso]

        subtotal = sum(mass_fracs[elem].values())
        for a in mass_fracs[elem]:
            mass_fracs[elem][a] /= subtotal

In [15]:
if False:  # 10/18/2023
    for iso in pp_and_cno_extras:
        elem, a = get_elem_and_a(iso)

        if elem not in lodders03 or a not in lodders03[elem]:
            print(iso)

In [16]:
# this version dumps missing isotopes of existing metals into heaviest

def make_mesa_mixture(col: str, initial_y: float = 0.2690, initial_z: float = 0.0187):
    initial_x = 1.0 - initial_y - initial_z

    for iso in ['h1', 'h2']:
        elem, a = get_elem_and_a(iso)
        print(iso, initial_x * lodders03[elem][a])

    for iso in ['he3', 'he4']:
        elem, a = get_elem_and_a(iso)
        print(iso, initial_y * lodders03[elem][a])

    metals = {}
    others = 0.0

    for elem, num_fracs in lodders03.items():
        if elem in ['H', 'He']: continue

        for a, num_frac in num_fracs.items():
            iso = f'{elem.lower()}{a}'
            mass_frac = 10.0 ** df.at[elem, col] *\
            lodders03[elem][a] * isotopes[iso]

            if iso in pp_and_cno_extras:
                metals[iso] = mass_frac
            else:
                others += mass_frac

    metals['mg24'] += others
    subtotal = sum(metals.values())
    for iso in metals:
        metals[iso] /= subtotal
        metals[iso] *= initial_z
        print(iso, metals[iso])

In [17]:
make_mesa_mixture('GS98')

h1 0.7122861813799999
h2 1.381862e-05
he3 4.464593000000001e-05
he4 0.26895535407
li7 1.071765477222455e-08
c12 0.0031814823599887643
c13 3.861917646278336e-05
n14 0.0009395413045252506
n15 3.7001349665027326e-06
o16 0.008734474685509457
o17 3.4614328704188498e-06
o18 1.9708580364796838e-05
f19 4.645188427716462e-07
ne20 0.001808711060787932
mg24 0.00396982602802655
